## 1. Setup

In [157]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from scipy.stats import chi2_contingency
import warnings
warnings.filterwarnings('ignore')

# Set random seed for reproducibility
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)

print("Libraries imported successfully")
print(f"Random seed: {RANDOM_SEED}")

Libraries imported successfully
Random seed: 42


## 1.1 Load and Preprocess Raw Survey Data

Clean the raw survey export file by removing metadata rows and mapping column names.

In [158]:
# Load raw survey export with text labels (skip first 2 metadata rows, use row 3 as header)
df_raw = pd.read_csv('../data/AIRS---AI-Readiness-Scale-labels.csv', skiprows=2)

print(f"Raw data loaded: {len(df_raw)} responses")
print(f"Raw columns: {len(df_raw.columns)}")
print(f"\nFirst 5 column names: {df_raw.columns[:5].tolist()}")

Raw data loaded: 325 responses
Raw columns: 54

First 5 column names: ['Session ID', 'Publish ID', 'Collector', 'Language', 'Start Date (America/Denver)']


In [159]:
# Column mapping from survey question names to variable codes
column_mapping = {
    'Duration (seconds)': 'Duration_seconds',
    'Progress': 'Progress',  # Keep for potential filtering
    'Performance Expectancy: How much do you agree with these statements about how AI tools help you get things done? | AI tools help me accomplish tasks more quickly': 'PE1',
    'Performance Expectancy: How much do you agree with these statements about how AI tools help you get things done? | Using AI improves the quality of my work or studies': 'PE2',
    'Effort Expectancy: How much do you agree with these statements about how easy AI tools are to learn and use? | Learning to use AI tools is easy for me': 'EE1',
    'Effort Expectancy: How much do you agree with these statements about how easy AI tools are to learn and use? | Interacting with AI tools is clear and understandable': 'EE2',
    'Social Influence: How much do you agree with these statements about the people around you and their views on using AI? | People whose opinions I value encourage me to use AI tools': 'SI1',
    'Social Influence: How much do you agree with these statements about the people around you and their views on using AI? | Leaders in my organization or school support the use of AI tools': 'SI2',
    'Facilitating Conditions: How much do you agree with these statements about the resources and support you have for using AI? | I have access to training or tutorials for the AI tools I use': 'FC1',
    'Facilitating Conditions: How much do you agree with these statements about the resources and support you have for using AI? | The AI tools I use are compatible with other tools or systems I use': 'FC2',
    'Hedonic Motivation (Perceived Enjoyment): How much do you agree with these statements about enjoyment when using AI tools? | Using AI tools is stimulating and engaging': 'HM1',
    'Hedonic Motivation (Perceived Enjoyment): How much do you agree with these statements about enjoyment when using AI tools? | AI tools make my work or studies more interesting': 'HM2',
    'Price Value: How much do you agree with these statements about whether using AI is worth your time and effort? | I get more value from AI tools than the effort they require': 'PV1',
    'Price Value: How much do you agree with these statements about whether using AI is worth your time and effort? | Using AI tools is worth the learning curve': 'PV2',
    'Habit: How much do you agree with these statements about your habits with AI tools? | Using AI tools has become a habit for me': 'HB1',
    'Habit: How much do you agree with these statements about your habits with AI tools? | I tend to rely on AI tools by default when I need help with tasks': 'HB2',
    'Voluntariness: How much do you agree with these statements about your freedom to choose whether or not to use AI tools? | I choose to use AI tools in my work because I find them helpful, not because I am required to': 'VO1',
    'Voluntariness: How much do you agree with these statements about your freedom to choose whether or not to use AI tools? | I could choose not to use AI tools in my work or studies if I preferred.': 'VO2',
    'Trust in AI: How much do you agree with these statements about trusting AI tools? | I trust AI tools to provide reliable information': 'TR1',
    'Trust in AI: How much do you agree with these statements about trusting AI tools? | I trust the AI tools that are available to me': 'TR2',
    'Explainability: How much do you agree with these statements about understanding how AI tools make their recommendations? | I understand how the AI tools I use generate their outputs': 'EX1',
    'Explainability: How much do you agree with these statements about understanding how AI tools make their recommendations? | I prefer AI tools that explain their recommendations': 'EX2',
    'Perceived Ethical Risk: How much do you agree with these statements about possible risks of AI? | I worry that AI tools could replace jobs in my field': 'ER1',
    'Perceived Ethical Risk: How much do you agree with these statements about possible risks of AI? | I am concerned about privacy risks when using AI tools': 'ER2',
    'AI Anxiety: How much do you agree with these statements about feeling uneasy or anxious about AI? | I feel uneasy about the increasing use of AI': 'AX1',
    'AI Anxiety: How much do you agree with these statements about feeling uneasy or anxious about AI? | I worry that I may be left behind if I do not keep up with AI': 'AX2',
    'AI Adoption Readiness: How much do you agree with these statements about your readiness to use AI? | I am ready to use more AI tools in my work or studies': 'BI1',
    'AI Adoption Readiness: How much do you agree with these statements about your readiness to use AI? | I would recommend AI tools to others': 'BI2',
    'AI Adoption Readiness: How much do you agree with these statements about your readiness to use AI? | I see AI as an important part of my future': 'BI3',
    'AI Adoption Readiness: How much do you agree with these statements about your readiness to use AI? | I plan to increase my use of AI tools in the next six months': 'BI4',
    'Usage Frequency: How often do you use the following AI tools? | Microsoft 365 Copilot or Microsoft Copilot': 'Usage_MSCopilot',
    'Usage Frequency: How often do you use the following AI tools? | ChatGPT': 'Usage_ChatGPT',
    'Usage Frequency: How often do you use the following AI tools? | Google Gemini': 'Usage_Gemini',
    'Usage Frequency: How often do you use the following AI tools? | Other AI tools (for example, Claude, Perplexity, Grok)': 'Usage_Other',
    'What is your current status?': 'Role',
    'What is your highest level of education completed?': 'Education',
    'Which industry or field best describes your primary area of work or study?': 'Industry',
    'How many years of work or study experience do you have in your field?': 'Experience',
    'Do you identify as a person with a disability (for example, vision, mobility, neurodivergence)?': 'Disability'
}

# Rename columns
df = df_raw.rename(columns=column_mapping)

# Select only the mapped columns (including Duration_seconds and Progress)
mapped_cols = list(column_mapping.values())
available_cols = [col for col in mapped_cols if col in df.columns]
df = df[available_cols].copy()

print(f"✓ Columns renamed and selected")
print(f"Cleaned dataset: {len(df)} rows × {len(df.columns)} columns")
print(f"Columns: {df.columns.tolist()[:10]}")

✓ Columns renamed and selected
Cleaned dataset: 325 rows × 39 columns
Columns: ['Duration_seconds', 'Progress', 'PE1', 'PE2', 'EE1', 'EE2', 'SI1', 'SI2', 'FC1', 'FC2']


In [160]:
# Convert duration from seconds to minutes
df['Duration_minutes'] = df['Duration_seconds'] / 60
df = df.drop('Duration_seconds', axis=1)

# Likert items are already numeric (1-5) in the export
likert_items = ['PE1', 'PE2', 'EE1', 'EE2', 'SI1', 'SI2', 'FC1', 'FC2', 
                'HM1', 'HM2', 'PV1', 'PV2', 'HB1', 'HB2', 'VO1', 'VO2',
                'TR1', 'TR2', 'EX1', 'EX2', 'ER1', 'ER2', 'AX1', 'AX2',
                'BI1', 'BI2', 'BI3', 'BI4']

# Usage frequency items are text labels in the export
# Map to numeric scale: Never=1, Rarely=2, Sometimes=3, Often=4, Daily=5
usage_items = ['Usage_MSCopilot', 'Usage_ChatGPT', 'Usage_Gemini', 'Usage_Other']

usage_mapping = {
    'Never': 1,
    'Rarely': 2,
    'Sometimes': 3,
    'Often': 4,
    'Daily': 5
}

for item in usage_items:
    df[item] = df[item].map(usage_mapping)

# Likert items are text labels in the export
# Map to numeric scale: Strongly disagree=1, Disagree=2, Neutral=3, Agree=4, Strongly agree=5
likert_mapping = {
    'Strongly disagree': 1,
    'Disagree': 2,
    'Neutral': 3,
    'Agree': 4,
    'Strongly agree': 5
}

for item in likert_items:
    df[item] = df[item].map(likert_mapping)

# Demographics are already text labels - clean up formatting for consistency
df['Role'] = df['Role'].str.strip()
df['Education'] = df['Education'].str.strip()
df['Experience'] = df['Experience'].str.strip()
df['Disability'] = df['Disability'].str.strip()

print("✓ Likert items converted from text labels to numeric (1-5)")
print("✓ Usage frequency items converted from text labels to numeric (1-5)")
print("✓ Demographic items validated (already text labels)")

✓ Likert items converted from text labels to numeric (1-5)
✓ Usage frequency items converted from text labels to numeric (1-5)
✓ Demographic items validated (already text labels)


In [161]:
# Calculate derived variables

# 1. AI Adoption (binary: 1 = any AI usage, 0 = no AI usage)
# Logic: If ALL usage == 1 (Never), then AI_Adoption = 0, else 1
df['AI_Adoption'] = ((df['Usage_MSCopilot'] > 1) | 
                     (df['Usage_ChatGPT'] > 1) | 
                     (df['Usage_Gemini'] > 1) | 
                     (df['Usage_Other'] > 1)).astype(int)

# 2. Total usage score (sum of all usage frequencies)
df['Total_Usage_Score'] = df[usage_items].sum(axis=1)

# 3. AI Adoption Level (categorical: None/Single/Multiple based on number of tools used)
tools_used = (df[usage_items] > 1).sum(axis=1)  # Count tools with usage > 1 (more than Never)
df['AI_Adoption_Level'] = tools_used.apply(lambda x: 'None' if x == 0 else 
                                                      'Single' if x == 1 else 
                                                      'Multiple')

# 4. Primary Tool (which tool is used most frequently)
tool_names = ['Microsoft 365 Copilot', 'ChatGPT', 'Gemini', 'Other AI Tool']
def get_primary_tool(row):
    if row['AI_Adoption'] == 0:
        return 'None'
    usage_cols = ['Usage_MSCopilot', 'Usage_ChatGPT', 'Usage_Gemini', 'Usage_Other']
    # Only consider tools with usage > 1 (more than Never)
    active_tools = {tool_names[i]: row[col] for i, col in enumerate(usage_cols) if row[col] > 1}
    if not active_tools:
        return 'None'
    return max(active_tools, key=active_tools.get)

df['Primary_Tool'] = df.apply(get_primary_tool, axis=1)

# 5. Experience Level (categorical based on years of experience)
# Label export uses different text: "1 to 3 years" vs "1-3 years"
experience_level_mapping = {
    'Less than 1 year': 'Entry',
    '1 to 3 years': 'Early',
    '4 to 6 years': 'Mid',
    '7 to 10 years': 'Senior',
    '11 or more years': 'Expert'
}
df['Experience_Level'] = df['Experience'].map(experience_level_mapping)

# 6. Work Context (map role + industry to professional/academic)
# Label export uses employment status + industry to determine academic affiliation
def map_work_context(row):
    role = row['Role']
    industry = row['Industry']
    
    if pd.isna(role):
        return 'Unknown'
    
    role_lower = str(role).lower()
    industry_lower = str(industry).lower() if pd.notna(industry) else ''
    
    # Students are Academic-Student
    if 'student' in role_lower:
        return 'Academic-Student'
    # Education industry workers who are employed are likely Academic-Faculty
    # Use 'employed -' (with space-dash) to match "Employed - *" but not "Not currently employed"
    elif 'education' in industry_lower and ('employed -' in role_lower or 'freelancer' in role_lower):
        return 'Academic-Faculty'
    # Everyone else is Professional
    else:
        return 'Professional'

df['Work_Context'] = df.apply(map_work_context, axis=1)

# 7. Usage Intensity (categorical based on total usage score)
# Scale: 1-5 per tool × 4 tools = range 4-20
# Thresholds: ≤4=Non-User, ≤8=Low, ≤12=Medium, >12=High
def map_usage_intensity(score):
    if score <= 4:  # All "Never" = 4
        return 'Non-User'
    elif score <= 8:  # Low usage
        return 'Low'
    elif score <= 12:  # Medium usage
        return 'Medium'
    else:  # High usage (13-20)
        return 'High'

df['Usage_Intensity'] = df['Total_Usage_Score'].apply(map_usage_intensity)

print("✓ Derived variables calculated:")
print(f"  - AI Adoption: {df['AI_Adoption'].sum()} adopters, {(df['AI_Adoption'] == 0).sum()} non-adopters")
print(f"  - Usage Intensity: {df['Usage_Intensity'].value_counts().to_dict()}")
print(f"  - Work Context: {df['Work_Context'].value_counts().to_dict()}")

✓ Derived variables calculated:
  - AI Adoption: 292 adopters, 33 non-adopters
  - Usage Intensity: {'Medium': 107, 'Low': 95, 'High': 90, 'Non-User': 33}
  - Work Context: {'Professional': 180, 'Academic-Student': 120, 'Academic-Faculty': 25}


In [162]:
# Reorder columns for better organization
column_order = ['Duration_minutes'] + likert_items + \
               ['Role', 'Education', 'Industry', 'Experience', 'Disability'] + \
               usage_items + \
               ['AI_Adoption', 'AI_Adoption_Level', 'Primary_Tool', 'Experience_Level', 
                'Work_Context', 'Usage_Intensity', 'Total_Usage_Score']

df = df[column_order]

# Save cleaned data
df.to_csv('../data/AIRS_clean.csv', index=False)

print("="*70)
print("✅ PREPROCESSING COMPLETE")
print("="*70)
print(f"Cleaned dataset: {len(df)} rows × {len(df.columns)} columns")
print(f"Saved to: ../data/AIRS_clean.csv")
print(f"\nReady for split-sample creation")

✅ PREPROCESSING COMPLETE
Cleaned dataset: 325 rows × 45 columns
Saved to: ../data/AIRS_clean.csv

Ready for split-sample creation


## 1.2 Load Cleaned Data for Split-Sample Creation

In [163]:
# Load preprocessed data
df = pd.read_csv('../data/AIRS_clean.csv')

print(f"Total sample: N = {len(df)}")
print(f"Variables: {len(df.columns)}")
print(f"\nFirst few columns: {df.columns[:10].tolist()}")

Total sample: N = 325
Variables: 45

First few columns: ['Duration_minutes', 'PE1', 'PE2', 'EE1', 'EE2', 'SI1', 'SI2', 'FC1', 'FC2', 'HM1']


## 2. Pre-Split Distributions

Document baseline distributions before splitting:

In [164]:
print("="*70)
print("PRE-SPLIT DISTRIBUTIONS")
print("="*70)

# Work Context
print("\n1. Work Context:")
work_context_dist = df['Work_Context'].value_counts()
print(work_context_dist)
print(f"Proportions: {(work_context_dist / len(df) * 100).round(1).to_dict()}")

# AI Adoption
print("\n2. AI Adoption:")
ai_adoption_dist = df['AI_Adoption'].value_counts()
print(f"Adopters (1): {ai_adoption_dist.get(1, 0)} ({ai_adoption_dist.get(1, 0)/len(df)*100:.1f}%)")
print(f"Non-Adopters (0): {ai_adoption_dist.get(0, 0)} ({ai_adoption_dist.get(0, 0)/len(df)*100:.1f}%)")

# Usage Intensity
print("\n3. Usage Intensity:")
usage_intensity_dist = df['Usage_Intensity'].value_counts()
print(usage_intensity_dist)

# Experience Level
print("\n4. Experience Level:")
experience_dist = df['Experience_Level'].value_counts()
print(experience_dist)

print("\n" + "="*70)

PRE-SPLIT DISTRIBUTIONS

1. Work Context:
Work_Context
Professional        180
Academic-Student    120
Academic-Faculty     25
Name: count, dtype: int64
Proportions: {'Professional': 55.4, 'Academic-Student': 36.9, 'Academic-Faculty': 7.7}

2. AI Adoption:
Adopters (1): 292 (89.8%)
Non-Adopters (0): 33 (10.2%)

3. Usage Intensity:
Usage_Intensity
Medium      107
Low          95
High         90
Non-User     33
Name: count, dtype: int64

4. Experience Level:
Experience_Level
Expert    110
Entry      65
Early      57
Mid        53
Senior     40
Name: count, dtype: int64



## 3. Create Stratified Split

Stratify by **Work Context × AI Adoption** to ensure balanced representation:

In [165]:
# Create stratification variable (Work Context × AI Adoption)
df['stratify_key'] = df['Work_Context'].astype(str) + '_' + df['AI_Adoption'].astype(str)

print("Stratification groups (Work Context × AI Adoption):")
print(df['stratify_key'].value_counts().sort_index())
print(f"\nTotal unique strata: {df['stratify_key'].nunique()}")

Stratification groups (Work Context × AI Adoption):
stratify_key
Academic-Faculty_0      3
Academic-Faculty_1     22
Academic-Student_0      5
Academic-Student_1    115
Professional_0         25
Professional_1        155
Name: count, dtype: int64

Total unique strata: 6


In [166]:
# Perform stratified 50/50 split
df_dev, df_holdout = train_test_split(
    df,
    test_size=0.50,
    stratify=df['stratify_key'],
    random_state=RANDOM_SEED
)

print("="*70)
print("SPLIT-SAMPLE CREATION COMPLETE")
print("="*70)
print(f"\nDevelopment sample: N = {len(df_dev)} ({len(df_dev)/len(df)*100:.1f}%)")
print(f"Holdout sample:     N = {len(df_holdout)} ({len(df_holdout)/len(df)*100:.1f}%)")
print(f"Total:              N = {len(df)}")
print("\n✓ Stratified by: Work Context × AI Adoption")
print(f"✓ Random seed: {RANDOM_SEED} (reproducible)")

SPLIT-SAMPLE CREATION COMPLETE

Development sample: N = 162 (49.8%)
Holdout sample:     N = 163 (50.2%)
Total:              N = 325

✓ Stratified by: Work Context × AI Adoption
✓ Random seed: 42 (reproducible)


## 4. Validation: Post-Split Balance

Verify stratification succeeded with chi-square tests:

In [167]:
def test_balance(variable_name, dev_sample, holdout_sample):
    """
    Test if variable distribution differs between development and holdout samples.
    H0: Distributions are equal (balanced split)
    """
    # Create proper contingency table
    dev_counts = dev_sample[variable_name].value_counts().sort_index()
    holdout_counts = holdout_sample[variable_name].value_counts().sort_index()
    
    contingency_table = pd.DataFrame({
        'Development': dev_counts,
        'Holdout': holdout_counts
    }).fillna(0)
    
    chi2, p_value, dof, expected = chi2_contingency(contingency_table)
    
    print(f"\n{variable_name}:")
    print(contingency_table)
    print(f"\nχ² = {chi2:.3f}, df = {dof}, p = {p_value:.4f}")
    
    if p_value > 0.05:
        print(f"✓ Balanced (p > 0.05): No significant difference between samples")
    else:
        print(f"⚠ Imbalanced (p ≤ 0.05): Significant difference detected")
    
    return p_value

print("="*70)
print("POST-SPLIT BALANCE TESTS (Chi-Square)")
print("="*70)

p_work_context = test_balance('Work_Context', df_dev, df_holdout)
p_ai_adoption = test_balance('AI_Adoption', df_dev, df_holdout)

POST-SPLIT BALANCE TESTS (Chi-Square)

Work_Context:
                  Development  Holdout
Work_Context                          
Academic-Faculty           13       12
Academic-Student           60       60
Professional               89       91

χ² = 0.059, df = 2, p = 0.9709
✓ Balanced (p > 0.05): No significant difference between samples

AI_Adoption:
             Development  Holdout
AI_Adoption                      
0                     17       16
1                    145      147

χ² = 0.000, df = 1, p = 0.9851
✓ Balanced (p > 0.05): No significant difference between samples


## 5. Descriptive Statistics: Compare Samples

Compare key Likert item means to ensure no systematic bias:

In [168]:
# Define construct items
construct_items = {
    'PE': ['PE1', 'PE2'],
    'EE': ['EE1', 'EE2'],
    'SI': ['SI1', 'SI2'],
    'FC': ['FC1', 'FC2'],
    'HM': ['HM1', 'HM2'],
    'PV': ['PV1', 'PV2'],
    'HB': ['HB1', 'HB2'],
    'VO': ['VO1', 'VO2'],
    'TR': ['TR1', 'TR2'],
    'EX': ['EX1', 'EX2'],
    'ER': ['ER1', 'ER2'],
    'AX': ['AX1', 'AX2'],
    'BI': ['BI1', 'BI2', 'BI3', 'BI4']
}

all_items = [item for items in construct_items.values() for item in items]

print("="*70)
print("LIKERT ITEM MEANS: Development vs. Holdout")
print("="*70)

comparison = pd.DataFrame({
    'Development_M': df_dev[all_items].mean(),
    'Development_SD': df_dev[all_items].std(),
    'Holdout_M': df_holdout[all_items].mean(),
    'Holdout_SD': df_holdout[all_items].std(),
    'Diff': df_dev[all_items].mean() - df_holdout[all_items].mean()
})

comparison['Abs_Diff'] = comparison['Diff'].abs()

print("\nSample of items (first 8):")
print(comparison.head(8).round(3))

print(f"\nMean absolute difference across all items: {comparison['Abs_Diff'].mean():.3f}")
print(f"Max absolute difference: {comparison['Abs_Diff'].max():.3f} ({comparison['Abs_Diff'].idxmax()})")

if comparison['Abs_Diff'].mean() < 0.10:
    print("\n✓ Excellent balance: Mean difference < 0.10 scale points")
elif comparison['Abs_Diff'].mean() < 0.20:
    print("\n✓ Good balance: Mean difference < 0.20 scale points")
else:
    print("\n⚠ Review: Mean difference ≥ 0.20 scale points")

LIKERT ITEM MEANS: Development vs. Holdout

Sample of items (first 8):
     Development_M  Development_SD  Holdout_M  Holdout_SD   Diff  Abs_Diff
PE1          3.660           1.154      3.571       1.122  0.090     0.090
PE2          3.340           1.201      3.325       1.217  0.014     0.014
EE1          3.809           1.013      3.736       0.987  0.072     0.072
EE2          3.654           1.036      3.577       1.024  0.078     0.078
SI1          3.136           1.187      3.037       1.201  0.099     0.099
SI2          3.383           1.143      3.282       1.130  0.101     0.101
FC1          3.272           1.195      3.209       1.214  0.063     0.063
FC2          3.444           1.092      3.429       1.018  0.015     0.015

Mean absolute difference across all items: 0.081
Max absolute difference: 0.186 (TR1)

✓ Excellent balance: Mean difference < 0.10 scale points


## 6. Save Split Samples

In [169]:
# Remove temporary stratification key
df_dev_clean = df_dev.drop(columns=['stratify_key'])
df_holdout_clean = df_holdout.drop(columns=['stratify_key'])

# Save to CSV
df_dev_clean.to_csv('../data/AIRS_clean_dev.csv', index=False)
df_holdout_clean.to_csv('../data/AIRS_clean_holdout.csv', index=False)

print("="*70)
print("SAMPLES SAVED")
print("="*70)
print(f"\n✓ Development sample: data/AIRS_clean_dev.csv")
print(f"  N = {len(df_dev_clean)}, Variables = {len(df_dev_clean.columns)}")
print(f"\n✓ Holdout sample: data/AIRS_clean_holdout.csv")
print(f"  N = {len(df_holdout_clean)}, Variables = {len(df_holdout_clean.columns)}")
print("\n✓ Stratification key removed from saved files")
print("\n" + "="*70)

SAMPLES SAVED

✓ Development sample: data/AIRS_clean_dev.csv
  N = 162, Variables = 45

✓ Holdout sample: data/AIRS_clean_holdout.csv
  N = 163, Variables = 45

✓ Stratification key removed from saved files


SAMPLES SAVED

✓ Development sample: data/AIRS_clean_dev.csv
  N = 162, Variables = 45

✓ Holdout sample: data/AIRS_clean_holdout.csv
  N = 163, Variables = 45

✓ Stratification key removed from saved files



## 7. Summary and Quality Checklist

In [170]:
print("="*70)
print("SPLIT-SAMPLE VALIDATION SUMMARY")
print("="*70)

checks = [
    ("Sample sizes approximately equal", abs(len(df_dev) - len(df_holdout)) <= 2),
    ("Development sample ≥ 150 (adequate for EFA)", len(df_dev) >= 150),
    ("Holdout sample ≥ 150 (adequate for CFA)", len(df_holdout) >= 150),
    ("Work Context balanced (p > 0.05)", p_work_context > 0.05),
    ("AI Adoption balanced (p > 0.05)", p_ai_adoption > 0.05),
    ("Mean item difference < 0.20", comparison['Abs_Diff'].mean() < 0.20),
    ("Files saved successfully", True)  # If we got here, it succeeded
]

print("\nQuality Checklist:")
for check, passed in checks:
    status = "✓" if passed else "✗"
    print(f"  {status} {check}")

all_passed = all(passed for _, passed in checks)

if all_passed:
    print("\n" + "="*70)
    print("✅ ALL VALIDATION CHECKS PASSED")
    print("="*70)
    print("\nReady for Phase 1: Exploratory Factor Analysis (EFA)")
    print("Next notebook: 01_EFA_Split_Sample_Development.ipynb")
else:
    print("\n" + "="*70)
    print("⚠ REVIEW REQUIRED")
    print("="*70)
    print("\nSome validation checks failed. Review results above.")

SPLIT-SAMPLE VALIDATION SUMMARY

Quality Checklist:
  ✓ Sample sizes approximately equal
  ✓ Development sample ≥ 150 (adequate for EFA)
  ✓ Holdout sample ≥ 150 (adequate for CFA)
  ✓ Work Context balanced (p > 0.05)
  ✓ AI Adoption balanced (p > 0.05)
  ✓ Mean item difference < 0.20
  ✓ Files saved successfully

✅ ALL VALIDATION CHECKS PASSED

Ready for Phase 1: Exploratory Factor Analysis (EFA)
Next notebook: 01_EFA_Split_Sample_Development.ipynb


---

## Notes

**Stratification Strategy**:
- Stratified by Work Context × AI Adoption (6 groups)
- Ensures balanced representation of key moderator variables
- Critical for moderation analysis (H4)

**Sample Size Adequacy**:
- N ≈ 159 per sample supports 12-factor CFA (N:q ratio ≈ 6.6:1 for 24 items)
- Minimum recommended: 5:1 for stable solutions (Bentler & Chou, 1987)
- Both samples adequate for planned analyses

**Random Seed**: 42 (reproducible splits for dissertation transparency)

**Next Steps**:
1. Run EFA on development sample (polychoric correlations)
2. Select items based on loadings ≥ 0.50, cross-loadings < 0.30
3. Test measurement model with CFA on holdout sample
4. Proceed to structural modeling if fit indices acceptable

---